# Simple gender analysis

- ## Imports

In [1]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import unicodedata

import os, sys, json, re, argparse, urllib2, html5lib
from os import listdir
from os.path import isfile, join
from bs4 import BeautifulSoup, Tag, UnicodeDammit
import pandas as pd
from pandas.io.json import json_normalize
import time
import math
from parse_script import *
from crawler import *

/home/duc-vinh/anaconda2/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


- ## Crawl

- ### Extract functions

In [2]:
path='../data/top100 per year/curated/'
col=['classement','titre','annee','annee_','categorie','realisateur','url','url_script']
titles=pd.DataFrame(columns=col)
for year in range(1995,2017):
    df=get_curated(year,path)
    df['annee_']=year
    titles = pd.concat([titles,df])

In [3]:
#retraiter la colonne catégorie
titles.reset_index(inplace=True,drop=True)
titles['categorie']=titles['categorie'].apply(lambda x : x.replace('\n','').replace(' ',''))
titles['categorie1'], titles['categorie2'], titles['categorie3']= titles['categorie'].str.split(',', 2).str

In [4]:
#consolider le dataframe pour les liens qui ont un lien script
titles = titles.dropna(subset=['url_script'],how='any')
titles.reset_index(inplace=True,drop=True)
titles

,annee,annee_,categorie,classement,duree,realisateur,titre,url,url_script,categorie1,categorie2,categorie3
0,(1995),1995.0,"Animation,Adventure,Comedy",1.0,81 min,John Lasseter,Toy Story,http://www.imsdb.com/Movie Scripts/Toy Story S...,http://www.imsdb.com/scripts/Toy-Story.html,Animation,Adventure,Comedy
1,(I) (1995),1995.0,"Adventure,Drama,History",3.0,140 min,Ron Howard,Apollo 13,http://www.imsdb.com/Movie Scripts/Apollo 13 S...,http://www.imsdb.com/scripts/Apollo-13.html,Adventure,Drama,History
2,(1995),1995.0,"Crime,Drama,Mystery",9.0,127 min,David Fincher,Se7en,http://www.imsdb.com/Movie Scripts/Se7en Scrip...,http://www.imsdb.com/scripts/Se7en.html,Crime,Drama,Mystery
3,(1995),1995.0,"Drama,Music",14.0,143 min,Stephen Herek,Mr. Holland's Opus,http://www.imsdb.com/Movie Scripts/Mr. Holland...,http://www.imsdb.com/scripts/Mr.-Holland's-Opu...,Drama,Music,NaN
4,(1995),1995.0,"Biography,Drama,History",18.0,178 min,Mel Gibson,Braveheart,http://www.imsdb.com/Movie Scripts/Braveheart ...,http://www.imsdb.com/scripts/Braveheart.html,Biography,Drama,History
5,(1995),1995.0,"Comedy,Crime,Thriller",19.0,105 min,Barry Sonnenfeld,Get Shorty,http://www.imsdb.com/Movie Scripts/Get Shorty ...,http://www.imsdb.com/scripts/Get-Shorty.html,Comedy,Crime,Thriller
6,(1995),1995.0,"Action,Crime,Drama",25.0,170 min,Michael Mann,Heat,http://www.imsdb.com/Movie Scripts/Heat Script...,http://www.imsdb.com/scripts/Heat.html,Action,Crime,Drama
7,(1995),1995.0,"Action,Comedy,Crime",28.0,119 min,Michael Bay,Bad Boys,http://www.imsdb.com/Movie Scripts/Bad Boys Sc...,http://www.imsdb.com/scripts/Bad-Boys.html,Action,Comedy,Crime
8,(1995),1995.0,"Comedy,Romance",32.0,97 min,Amy Heckerling,Clueless,http://www.imsdb.com/Movie Scripts/Clueless Sc...,http://www.imsdb.com/scripts/Clueless.html,Comedy,Romance,NaN
9,(1995),1995.0,"Drama,Romance",39.0,136 min,Ang Lee,Sense and Sensibility,http://www.imsdb.com/Movie Scripts/Sense and S...,http://www.imsdb.com/scripts/Sense-and-Sensibi...,Drama,Romance,NaN


In [5]:
print('Only remains %s movies.'%(len(titles)))

Only remains 367 movies.


In [6]:
#les différentes catégories:
l=list(titles.categorie1)+list(titles.categorie2)+list(titles.categorie3)
set(l)

{nan,
 u'Action',
 u'Adventure',
 u'Animation',
 u'Biography',
 u'Comedy',
 u'Crime',
 u'Drama',
 u'Family',
 u'Fantasy',
 u'History',
 u'Horror',
 u'Music',
 u'Musical',
 u'Mystery',
 u'Romance',
 u'Sci-Fi',
 u'Sport',
 u'Thriller',
 u'War',
 u'Western'}

- ### Analysis Functions

In [7]:
def clean_character(char):
    if isinstance(char,unicode) or isinstance(char,str):
        char=char.replace('\n','').replace('*','').replace('O/S','').replace('V.O.','')
        char=re.sub(' +',' ',char)
        if 'CONTINUED' in char:
            return u'CONTINUED'
        if '(' in char or ')' in char:
            regex = re.compile(".*?\((.*?)\)")
            to_delete = re.findall(regex, char)
            for e in to_delete:
                char=char.replace('(%s)' %e,'')
            if char=='' or ')'in char:
                return None
            else:
                return char.rstrip() #delete lose end white space
        else:
            return char.rstrip()
    else:
        return None
    
def continued_speech(char_vector):
    char_vector=list(char_vector)
    while 'CONTINUED' in char_vector:
        count=0
        idx=char_vector.index("CONTINUED")
        for element in char_vector[idx-1::-1]:
            #if (isinstance(element,str) or isinstance(element,unicode)) and (element!='unknown'):
            if element and element!='unknown':
                char_vector[idx]=char_vector[idx-1-count]
                #print('Replaces CONTINUED with %s' %(char_vector[idx-1-count]))
                break
            count=count+1
            if count==len(char_vector[idx-1::-1]):
                char_vector[idx]='unknown'
                break
    return char_vector


# to remove outliers
def principal_characters(char_dic):
    mean = 0
    for i, j in char_dic.items():
        mean = mean + j
    mean = mean / len(char_dic)

    stddev = 0
    for i, j in char_dic.items():
        stddev = stddev + (j - mean)*(j - mean)
    stddev = stddev / len(char_dic)
    stddev = math.sqrt(stddev)

    temp = {}
    for i, j in char_dic.items():
        if (math.fabs(j - mean) < 3*stddev):
            temp[i] = j
    print(stddev,mean)
    return temp


def char_analysis(df):
    d=dict(Counter(df['character_clean']))
    d={key: value for key, value in d.items() 
                 if key}
    charlist = sorted(d)
    gender = dict()
    lines=[element.split('\n') for element in df['text']]
    lines=[item for sublist in lines for item in sublist]

    for char in charlist:
        m, f, p = 0, 0, 0 
        for i in range(len(lines)):
            srch = re.search(char, lines[i], re.I)
            if not srch:
                continue
            if srch.group().isupper():
                continue
            else:
                k, k_range = -1, 2
                while k + i + 1 < len(lines):
                    k = k + 1
                    m = m + len(re.findall("\s+he", lines[k+i], re.I)) + len(re.findall("\s+him", lines[k+i], re.I))
                    f = f + len(re.findall("\s+she", lines[k+i], re.I)) + len(re.findall("\s+her", lines[k+i], re.I))
                    if lines[k+i].isupper():
                        break
                    if k >= k_range:
                        break
                k, k_range = -1, 4
                while k + i + 1 < len(lines):
                    k = k + 1
                    m1, m2, m3, m4 = re.search("\s+he", lines[k+i], re.I), re.search("\s+him", lines[k+i], re.I),\
                                    re.search("\s+she", lines[k+i], re.I), re.search("\s+her", lines[k+i], re.I)
                    if m1:
                        if m2:
                            mc = min(m1.span()[0], m2.span()[0])
                        else:
                            mc = m1.span()[0]
                    else:
                        mc = len(lines[k+i])
                    if m3:
                        if m4:
                            fc = min(m3.span()[0], m4.span()[0])
                        else:
                            fc = m3.span()[0]
                    else:
                        fc = len(lines[k+i])
                    if not(m1 or m2 or m3 or m4):
                        continue
                    else:
                        if (mc < fc):
                            p = p - 1
                        else:
                            p = p + 1
                        break
                    if lines[k+i].isupper():
                        break
                    if k >= k_range:
                        break
        gender[char] = m, f, p

    for char, g in gender.items():
        g_score = 2*g[1] - g[0] 
        g_sum = g[1]+g[0]
        g_score = g_score
        if (g_sum == 0.0) :#or (0 == 1):
            #print(char+" Undetermined ")
            gender[char] = "?" #0, 0, -1
        elif g_score > 0:
            #print(char+" Female "+str(g[0])+","+str(g[1])+","+str(g[2])+","+str(g_score))
            gender[char] = "F" #0, 0, 1
        else:
            #print(char+" Male", " ", str(g[0])+","+str(g[1])+","+str(g[2])+","+str(g_score))
            gender[char] = "M" #0, 0, 0
            
    return d,gender

- ### Script loader and parser

In [26]:
def get_all_bygenre(genre,titles):
    urls = list(titles[(titles['categorie1']==genre) | (titles['categorie2']==genre)]['url_script'])
    output=[]
    nb_of_errors=0
    for url in urls:
        path='../data/script/%s/' %(genre.lower())
        name=url.replace(':','').replace('/','')
        name_ = titles[titles['url_script']==url]['titre']
        name_ = list(name_)[0]
        try:
            test=parse(url,path+'raw/',name_)
            try:
                #print(df['text'][0].split('\n')[0])
                vec=test['character'].apply(lambda x :clean_character(x))
                test['character_clean']=continued_speech(vec)
                char,gender=char_analysis(test)
                #output_actions_.append(char)
                #create gender column in dataframe:
                test['gender']=[gender.get(item,item)  for item in list(test['character_clean'])]
                if not os.path.exists(path+'clean/'):
                    os.makedirs(path+'clean/')
                test.to_csv(path+'clean/'+name_+'.csv',sep='|',encoding='latin1')
            except Exception as e:
                print('------Caught exception : %s' %e)
                nb_of_errors=nb_of_errors+1
            output.append(test)
        except Exception as e:
            print(e)
            nb_of_errors=nb_of_errors+1
    return output

def load_bygenre(genre):
    genre=genre.lower()
    mypath = "../data/script/%s/clean/" %(genre)
    onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    output=[]
    for element in onlyfiles:
        output.append(pd.read_csv(mypath+element,sep='|',encoding='latin1',index_col=0))
    return output

In [ ]:
liste=[ u'Action',
 u'Adventure',
 u'Animation',
 u'Biography',
 u'Comedy',
 u'Crime',
 u'Drama',
 u'Family',
 u'Fantasy',
 u'History',
 u'Horror',
 u'Music',
 u'Musical',
 u'Mystery',
 u'Romance',
 u'Sci-Fi',
 u'Sport',
 u'Thriller',
 u'War',
 u'Western']

for element in liste:
    get_all_bygenre(element,titles)

Getting script @ http://www.imsdb.com/scripts/Heat.html.
This script is too unstable to parse
      Done parsing script at http://www.imsdb.com/scripts/Heat.html in 2.31809091568
-----------------
Getting script @ http://www.imsdb.com/scripts/Bad-Boys.html.
This script is too unstable to parse
      Done parsing script at http://www.imsdb.com/scripts/Bad-Boys.html in 2.15413808823
-----------------
Getting script @ http://www.imsdb.com/scripts/Mighty-Morphin-Power-Rangers-The-Movie.html.
      Done parsing script at http://www.imsdb.com/scripts/Mighty-Morphin-Power-Rangers-The-Movie.html in 1.69972109795
-----------------
------Caught exception : unbalanced parenthesis
Getting script @ http://www.imsdb.com/scripts/Judge-Dredd.html.
      Done parsing script at http://www.imsdb.com/scripts/Judge-Dredd.html in 2.03622293472
-----------------
------Caught exception : unbalanced parenthesis
Found a <pre> inside the <pre>
Getting script @ http://www.imsdb.com/scripts/Assassins.html.
      D

In [ ]:
#plot year evolution

In [ ]:
thriller = load_bygenre('Thriller')

In [ ]:
m=sum(male_nb_speech)
f=sum(female_nb_speech)

 
# Data to plot
labels = ["M","F"]
sizes = [m, f]
colors = ['lightskyblue', 'pink']
plt.figure(figsize=(8,8))
plt.pie(sizes, labels=labels, colors=colors,
        autopct='%1.1f%%', shadow=True, startangle=140)
plt.title("Repartition des repliques dans les films d'actions") 
plt.axis('equal')
plt.show()

In [ ]:
i=1
male_nb_speech=len(output_actions[i][output_actions[i]['gender']=='M'])
female_nb_speech=len(output_actions[i][output_actions[i]['gender']=='F'])

 
# Data to plot
labels = ["M","F"]
sizes = [male_nb_speech, female_nb_speech]
colors = ['lightskyblue', 'pink']
plt.figure(figsize=(8,8))
plt.pie(sizes, labels=labels, colors=colors,
        autopct='%1.1f%%', shadow=True, startangle=140)
plt.title('Repartition des repliques dans %s' %output_actions[i]['text'][0].split('\n')[0]) 
plt.axis('equal')
plt.show()

In [ ]:
i=10
male_nb_speech=len(output_actions[i][output_actions[i]['gender']=='M'])
female_nb_speech=len(output_actions[i][output_actions[i]['gender']=='F'])

 
# Data to plot
labels = ["M","F"]
sizes = [male_nb_speech, female_nb_speech]
colors = ['lightskyblue', 'pink']
plt.figure(figsize=(8,8))
plt.pie(sizes, labels=labels, colors=colors,
        autopct='%1.1f%%', shadow=True, startangle=140)
plt.title('Repartition des repliques dans %s' %output_actions[i]['text'][0].split('\n')[0]) 
plt.axis('equal')
plt.show()

In [ ]:
%%time
romances_url = list(titles[(titles['categorie1']=='Romance') | (titles['categorie2']=='Romance')]['url_script'])
output_romances=[]
nb_of_errors=0
for url in romances_url:
    path='../data/script/romance/'
    name=url.replace(':','').replace('/','')
    try:
        test=parse(url,path+'raw/',name)
        try:
            #print(df['text'][0].split('\n')[0])
            vec=test['character'].apply(lambda x :clean_character(x))
            test['character_clean']=continued_speech(vec)
            char,gender=char_analysis(test)
            #output_actions_.append(char)
            #create gender column in dataframe:
            test['gender']=[gender.get(item,item)  for item in list(test['character_clean'])]
            if not os.path.exists(path+'clean/'):
                os.makedirs(path+'clean/')
            test.to_csv(path+'clean/'+name+'.csv',sep='|',encoding='latin1')
        except Exception as e:
            print('------Caught exception : %s' %e)
            nb_of_errors=nb_of_errors+1
        output_romances.append(test)
    except Exception as e:
        print(e)
        nb_of_errors=nb_of_errors+1

In [ ]:
print('The number of errors is %s out of a total %s.' %(nb_of_errors,len(romances_url)))

In [ ]:
#construct character info and add it to dataframe
output_romances_=[]
male_nb_speech=[]
female_nb_speech=[]
for df in output_romances:
    try:
        #print(df['text'][0].split('\n')[0])
        test=df['character'].apply(lambda x :clean_character(x))
        df['character_clean']=continued_speech(test)
        char,gender=char_analysis(df)
        output_romances_.append(char)
        #create gender column in dataframe:
        df['gender']=[gender.get(item,item)  for item in list(df['character_clean'])]
        male_nb_speech.append(len(df[df['gender']=='M']))
        female_nb_speech.append(len(df[df['gender']=='F']))
    except Exception as e:
        print('------Caught exception : %s' %e)
        nb_of_errors=nb_of_errors+1

In [ ]:
print('The number of errors is %s out of a total %s.' %(nb_of_errors,len(romances_url)))

In [ ]:
m=sum(male_nb_speech)
f=sum(female_nb_speech)

 
# Data to plot
labels = ["M","F"]
sizes = [m, f]
colors = ['lightskyblue', 'pink']
plt.figure(figsize=(8,8))
plt.pie(sizes, labels=labels, colors=colors,
        autopct='%1.1f%%', shadow=True, startangle=140)
plt.title("Repartition des repliques dans les films romantiques") 
plt.axis('equal')
plt.show()

In [ ]:
m=len(actions_url)
f=len(romances_url)

 
# Data to plot
labels = ["Actions","Romances"]
sizes = [m, f]
colors = ['lightskyblue', 'pink']
plt.figure(figsize=(8,8))
plt.pie(sizes, labels=labels, colors=colors,
        autopct='%1.1f%%', shadow=True, startangle=140)
plt.title("Ratio Actions-Romances parmis les top100") 
plt.axis('equal')
plt.show()

In [ ]:
def find(df,to_find):
    count=0
    for element in df['text']:
        if to_find in normalize_text(element):
            count+=1
    return count

def find_female_in(df,to_find):
    count_female=0
    count_total=0
    idx=0
    try:
        list_female=list(df[df['gender']=='F']['character_clean'].apply(lambda x : normalize_text(x)))
        list_female=list(set(list_female))
        #list_female.append('she').append('her')
    except Exception as e:
        try:
            
            for element in df['text']:
                if to_find in normalize_text(element):
                    count_total+=1
                    if 'she' in element or "her" in element:
                        count_female+=1
            return count_female,count_total
        except Exception as ex:
            print('Warning : %s.' %(ex))
            return 0,0
    list_female.append('she')
    list_female.append("her")
    for element in df['text']:
        if to_find in normalize_text(element):
            count_total+=1
            for indicator in list_female: #she or her
                if indicator in element:
                    count_female+=1
                    break
        idx+=1
    return count_female,count_total

def find_any(df,l):
    for element in df['text']:
        for room in l:
            if room in normalize_text(element):
                print("Found %s" %room)
    return

def normalize_text(text):
    text = text.lower()
    normal = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore')
    return normal

In [ ]:
l=['KITCHEN','Kitchen','kitchen','bed','BED','BEDROOM','Bed','Bedroom','bedroom','BATH','bath','Bath','BATHROOM',
  'Bathroom','bathroom','toilet','TOILET','Toilet']
l_norm=['kitchen','bed','bedroom','bath','bathroom','toilet']

In [ ]:
import random
kitchen_count_action=0
kitchen_count_romance=0
rand_idx = random.sample(range(0,len(output_actions)),  len(output_romances))
for df in [output_actions[i] for i in rand_idx]:
    try:
        kitchen_count_action += find(df,'kitchen')
    except Exception as e:
        print(e)
for df in output_romances:
    try:
        kitchen_count_romance += find(df,'kitchen')
    except Exception as e:
        print(e)

In [ ]:
# Data to plot
labels = ["in actions","in romance"]
sizes = [kitchen_count_action, kitchen_count_romance]
colors = ['lightskyblue', 'pink']
plt.figure(figsize=(8,8))
plt.pie(sizes, labels=labels, colors=colors,
        autopct='%1.1f%%', shadow=True, startangle=140)
plt.title("Repartition de la presence des cuisines dans les films actions-romances") 
plt.axis('equal')
plt.show()

print('Actions: %s de présence cuisines' %(kitchen_count_action))
print('Romances: %s  de présence cuisines' %(kitchen_count_romance))

In [ ]:
kitchen_count_action_f=0
kitchen_count_action =0
kitchen_count_romance_f=0
kitchen_count_romance = 0
for df in output_actions:
    kitchen_count_action_f += find_female_in(df,'kitchen')[0]
    kitchen_count_action += find_female_in(df,'kitchen')[1]
for df in output_romances:
    kitchen_count_romance_f += find_female_in(df,'kitchen')[0]
    kitchen_count_romance += find_female_in(df,'kitchen')[1]

In [ ]:
# Data to plot
labels = ["in actions","in romance"]
sizes = [kitchen_count_action_f, kitchen_count_romance_f]
colors = ['lightskyblue', 'pink']
plt.figure(figsize=(8,8))
plt.pie(sizes, labels=labels, colors=colors,
        autopct='%1.1f%%', shadow=True, startangle=140)
plt.title("Repartition de la presence des cuisines dans les films actions-romances") 
plt.axis('equal')
plt.show()

print('Actions: %s femmes & cuisines' %(kitchen_count_action_f))
print('Romances: %s femmes & cuisines' %(kitchen_count_romance_f))

In [ ]:
# Data to plot
labels = ["other","women"]
sizes = [kitchen_count_action - kitchen_count_action_f, kitchen_count_action_f]
colors = ['lightskyblue', 'pink']
plt.figure(figsize=(8,8))
plt.pie(sizes, labels=labels, colors=colors,
        autopct='%1.1f%%', shadow=True, startangle=140)
plt.title("Repartition H/F & cuisne dans les films d'actions") 
plt.axis('equal')
plt.show()

print(' %s femmes & cuisines' %(kitchen_count_action_f))
print('%s autres & cuisines' %(kitchen_count_action- kitchen_count_action_f))

In [ ]:
# Data to plot
labels = ["other","women"]
sizes = [kitchen_count_romance - kitchen_count_romance_f, kitchen_count_romance_f]
colors = ['lightskyblue', 'pink']
plt.figure(figsize=(8,8))
plt.pie(sizes, labels=labels, colors=colors,
        autopct='%1.1f%%', shadow=True, startangle=140)
plt.title("Repartition H/F & cuisine dans les films d'actions") 
plt.axis('equal')
plt.show()

print('Actions: %s femmes & cuisines' %(kitchen_count_action_f))
print('Romances: %s femmes & cuisines' %(kitchen_count_romance_f))